Code to impose the the final tumor image on the brain image

In [ ]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/

In [ ]:
import tensorflow as tf
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'
block_h = 32
block_w = 32
Model = tf.keras.models.load_model(path + 'Outputs/' + 'Model_Case_1_1.h5')

In [ ]:
def window(p, q, layer, flair_data):
    W = flair_data[p:p+32, q:q+32, layer]
    return W


def ML_function(Windows, layer):
    count = 0
    X = np.array(Windows).reshape(-1, block_h, block_w, 1)
    predicted_classes = np.argmax(Model(X), axis=1)
    for val in predicted_classes:
        if (val == 1):
            count = count + 1
    #print(count)
    return count


def fun(a, b, layer, flair_data):
    if (a < 0 or a > 239 or b < 0 or b > 239):
        print("invalid point")
    else:
        Windows = []
        for x in range(a-31, a+1):
            for y in range(b-31, b+1):
                Windows.append(window(x, y, layer, flair_data))
        count_1 = ML_function(Windows, layer)
    return (count_1/1024)


In [ ]:
def Generate_Weighted_Matrix():
    import tqdm
    import pickle
    taskdir = os.listdir(Task_data)
    if '.DS_Store' in taskdir:
        taskdir.remove('.DS_Store')
    for Type in taskdir:
        print(Type)
        Type_Path = os.path.join(Task_data, Type + '/')
        typepathdir = os.listdir(Type_Path)
        if '.DS_Store' in typepathdir:
            typepathdir.remove('.DS_Store')
            print
        for patient in typepathdir:
            print(patient)
            Patient_Path = os.path.join(Type_Path, patient + '/')
            os.chdir(Patient_Path)
            flair = nib.load('{}_flair.nii.gz'.format(patient))
            flair_data = flair.get_fdata()  # Converting nii to 3d np array

            # Scale data between 0 & 255
            for l in range(0, flair_data.shape[2]):
                if (flair_data[:, :, l].max() == 0):
                    continue
                else:
                    flair_data[:, :, l] *= (255.0/flair_data[:, :, l].max())

            seg_mask = nib.load('{}_seg.nii.gz'.format(patient))
            seg_mask_data = seg_mask.get_fdata()  # Converting nii to 3d np array

            #Extracting only those layers from mask which have non zero values
            z = np.any(seg_mask_data, axis=(0, 1))
            nonzero_layers = np.nonzero(z)
            nonzero_layer_indices = nonzero_layers[0]
            num_layers = nonzero_layer_indices.size
            #print(num_layers)
            arr = np.zeros((240, 240, num_layers))
            layer_num = 0

            #Finding indices of cancer pixels using mask data
            for layer in nonzero_layer_indices:
                indices = np.transpose(np.nonzero(seg_mask_data[:, :, layer]))
                #print(indices, layer)
                for [a, b] in indices:
                    arr[a, b, layer_num] = fun(a, b, layer, flair_data)
                layer_num = layer_num + 1

            #print(np.count_nonzero(np.any(arr, axis=(0, 1))))

            #Calculating weighted average
            Weighted_matrix = np.zeros((240, 240))
            for x in range(0, 240):
                for y in range(0, 240):
                    Sum = 0
                    num_terms = 0
                    for slice_num in range(0, num_layers):
                        if (arr[x][y][slice_num] != 0):
                            Sum = Sum + arr[x][y][slice_num]
                            num_terms = num_terms + 1
                    if (num_terms != 0):
                        Weighted_matrix[x][y] = Sum/num_terms
            print(Weighted_matrix.shape)

            #Storing Wiegted Average Matrix of the patient
            pickle_out = open(
                Patient_Path + '/W_M_{}.pickle'.format(patient), 'wb')
            pickle.dump(Weighted_matrix, pickle_out)
            pickle_out.close()


In [ ]:
# All Function Calls
import os
import numpy as np
import nibabel as nib

PATH = path + 'Data/'
Task_data = PATH + 'Task_data/'

# Function Call --> Generate Tumor Images of Patients
Generate_Weighted_Matrix()


In [ ]:
import pickle
pickle_in_negative = open(
    '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/Task_data/MGMT_negative/BraTS2021_00003/W_M_BraTS2021_00003.pickle', 'rb')
W_M_Negative_3 = pickle.load(pickle_in_negative)


In [ ]:
pickle_in_positive = open(
    '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/Task_data/MGMT_positive/BraTS2021_00000/W_M_BraTS2021_00000.pickle', 'rb')
W_M_Positive_0 = pickle.load(pickle_in_positive)


In [ ]:
from copy import copy, deepcopy
print(W_M_Negative_3)
x = deepcopy(W_M_Negative_3)
print(x)

for i in range(len(x)):
    for j in range(len(x[0])):
        if x[i][j] == 0:
            pass
        else:
            x[i][j] = abs(x[i][j]-1)

x


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 15))
# plt.spy(W_M_Negative_3, markersize=1)
plt.imshow(x, cmap="Reds")


In [ ]:
import numpy as np
np.nonzero(x)

In [ ]:
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'

CATEGORIES = ['MGMT_negative', 'MGMT_positive']

PATH = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/Original_Data_Backup/MGMT_negative/BraTS2021_00003/BraTS2021_00003_flair.nii.gz'


In [ ]:
import nibabel as nib
def maxpixels():
    import os
    print("Fetching image with maximum image size")
    flair = nib.load(PATH)
    flair_data= flair.get_fdata() # coverting nii to 3d np array
    seg_mask = nib.load(
        '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/Original_Data_Backup/MGMT_negative/BraTS2021_00003/BraTS2021_00003_seg.nii.gz')
    seg_mask_data = seg_mask.get_fdata()  # coverting nii to 3d np array
    #Extracting only those layers from mask hich have non zero values
    z = np.any(seg_mask_data, axis=(0, 1))
    # zmin & zmax saves the corresponding layer numbers of tumor regions
    zmin, zmax = np.where(z)[0][[0, -1]]
    print(zmin, zmax)
    # print(z)
    # Figuring out layer with maximum pixels
    layer=0
    max = np.nonzero(flair_data[77])[0].max()
    print(np.nonzero(flair_data[77])[0][0])
    # for i in range(zmin,zmax):
    #     for j in range(2):
    #         max1 = np.nonzero(flair_data[i])[j].max()
    #         if max <max1:
    #             max = max1
    #             layer=i
    #         else:
    #             pass
        # print(i)
        # print(flair_data[i])
        # print(np.nonzero(flair_data[i]))

    # for i in range(zmin,zmax):
    #     for 
        
    print(layer)
    print(np.nonzero(flair_data[layer]))
    # print("here",len(np.nonzero(flair_data[77])))
    print(np.nonzero(flair_data[129])[1].max())
    return flair_data[layer]

        


In [ ]:
y= maxpixels()
y

In [ ]:
import matplotlib.pyplot as plt

flair = nib.load(PATH)
flair_data = flair.get_fdata()
plt.figure(figsize=(15, 15))
# plt.spy(W_M_Negative_3, markersize=1)
plt.imshow(flair_data[129])


In [ ]:
import seaborn as sns
import numpy as np
import nibabel as nib
# import itk
# import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('darkgrid')

In [ ]:
image_path_flair = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/Original_Data_Backup/MGMT_negative/BraTS2021_00003/BraTS2021_00003_flair.nii.gz'
image_path_seg = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/Original_Data_Backup/MGMT_negative/BraTS2021_00003/BraTS2021_00003_seg.nii.gz'


In [ ]:
image_obj_flair = nib.load(image_path_flair)
image_obj_seg = nib.load(image_path_seg)


In [ ]:
image_data_flair = image_obj_flair.get_fdata()
image_data_seg = image_obj_seg.get_fdata()


In [ ]:
image_data_flair.shape

In [ ]:
image_data_flair

In [ ]:
from copy import copy, deepcopy
zmin = 77
zmax = 136
max_pixel=0
layer=0
for i in range(zmin,zmax):
    for j in range(240):
        for k in range(240):
            blank = np.zeros((240, 240))
            blank[j][k]= flair_data[j][k][i]
            # print(np.nonzero(blank))
            blank_size=len(np.nonzero(blank)[0])
            blank_size1=len(np.nonzero(blank)[1])
            print(blank_size)
            # if blank_size>max_pixel:
            #     max_pixel=blank_size
            #     layer =i
print(layer)

In [ ]:
blank = np.zeros((240,240))
blank

In [ ]:
flair_data.shape
